In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import pandas as pd

# Assuming there are N classes to predict
N_CLASSES = 42

model = models.Sequential([
    layers.Input(shape=(7,)),  # 7 features: accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, ind
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(N_CLASSES, activation='softmax')  # Softmax for multi-class classification
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if one-hot encoded
              metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 42)                2730      
                                                                 
Total params: 7,402
Trainable params: 7,402
Non-trainable params: 0
_________________________________________________________________


In [2]:
def extract_activity_and_status(filename):
    # Extract the parts of the filename
    filename_without_extension = os.path.splitext(os.path.basename(filename))[0]
    details = filename_without_extension.split("_")
    
    # Extract the device name, ID, activity, and timestamp from the details
    device_name = details[0]
    user_id = details[1]
    activity = details[2]
    activity_sub = details[3]
    status = details[4]

    return activity, activity_sub, status

In [8]:
input_directory = "../assets/pdiot_submission/s2102187"

data_list = []
label_list = []
total_rows = 0

for file in os.listdir(input_directory):
    if file.endswith(".csv") and "unprocessed" not in file:
        filename = os.path.join(input_directory, file)
        label, _, _ = extract_activity_and_status(filename)

        df = pd.read_csv(filename, usecols=[1,2,3])

        # Determine the number of rows to take from this file
        rows_to_take = min(700 - total_rows, len(df))
        
        # Update the df to only contain the necessary rows and update our counter
        df = df.head(rows_to_take)
        label_list.append(label)
        data_list.append(df)

# all_data = pd.concat(data_list, ignore_index=True)
print(data_list)

[      accel_x   accel_y   accel_z
0    0.148682 -1.047180 -0.022522
1    0.259766 -1.153625 -0.092590
2    0.283447 -1.151672 -0.052063
3    0.201660 -0.957581 -0.007629
4    0.179443 -0.877014  0.004089
..        ...       ...       ...
695  0.348877 -0.735168 -0.317200
696  0.275391 -0.732239 -0.240051
697  0.243164 -0.618225 -0.219299
698  0.208008 -0.619934 -0.167542
699  0.140137 -0.737854 -0.178284

[700 rows x 3 columns],       accel_x   accel_y   accel_z
0   -0.030518 -0.773743 -0.074768
1   -0.083252 -0.751770  0.013611
2   -0.171875 -0.845032 -0.005920
3   -0.116211 -0.729797 -0.059387
4   -0.157959 -0.684875 -0.076965
..        ...       ...       ...
695  0.140625 -0.706360 -0.010315
696  0.106934 -1.006165 -0.203918
697 -0.081055 -1.255920 -0.286438
698 -0.003174 -1.221741 -0.334290
699 -0.263672 -2.000000 -0.533997

[700 rows x 3 columns],       accel_x   accel_y   accel_z
0   -0.363770 -0.086975  0.950378
1   -0.388916 -0.085510  0.945007
2   -0.421631 -0.103577  0.8419

In [9]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Normalizing sensor readings
scaler = StandardScaler()

sensor_columns = ['accel_x', 'accel_y', 'accel_z']

scaler.fit(data_list[0][sensor_columns])

for i in range(len(data_list)):
    data_list[i][sensor_columns] = scaler.transform(data_list[i][sensor_columns])


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert data and labels to numpy arrays
X = np.array([df.values for df in data_list])
X = np.stack(X)
y = LabelEncoder().fit_transform(label_list)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert X_train to float32
X_train = X_train.astype('float32')

# Define LSTM model
model = models.Sequential([
    layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    layers.Dense(len(np.unique(y)), activation='relu')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2) 


Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 7.3980 - accuracy: 0.1143 - val_loss: 7.6500 - val_accuracy: 0.1111
Epoch 2/50
1/1 [==============================] - 0s 169ms/step - loss: 5.8097 - accuracy: 0.1429 - val_loss: 9.7965 - val_accuracy: 0.1111
Epoch 3/50
1/1 [==============================] - 0s 168ms/step - loss: 5.5525 - accuracy: 0.1429 - val_loss: 9.7582 - val_accuracy: 0.1111
Epoch 4/50
1/1 [==============================] - 0s 166ms/step - loss: 5.4471 - accuracy: 0.2000 - val_loss: 9.7173 - val_accuracy: 0.1111
Epoch 5/50
1/1 [==============================] - 0s 166ms/step - loss: 5.3667 - accuracy: 0.1714 - val_loss: 9.6770 - val_accuracy: 0.1111
Epoch 6/50
1/1 [==============================] - 0s 169ms/step - loss: 5.2975 - accuracy: 0.2286 - val_loss: 9.6368 - val_accuracy: 0.2222
Epoch 7/50
1/1 [==============================] - 0s 167ms/step - loss: 5.2323 - accuracy: 0.3714 - val_loss: 9.5944 - val_accuracy: 0.2222
Epoch 8/50
1/1 [=======

In [13]:
print(model.summary)

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x00000179DF3ADC88>>


In [14]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

# tflite_model_test = converter.convert()

# with open("model.tflite", "wb") as f:
#     f.write(tflite_model_test)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Allow for TensorFlow ops that aren't natively supported in TFLite
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# Disable the lowering of tensor list operations
converter._experimental_lower_tensor_list_ops = False

tflite_model_test = converter.convert()

with open("model_test.tflite", "wb") as f:
    f.write(tflite_model_test)


INFO:tensorflow:Assets written to: C:\Users\XZHANG~1\AppData\Local\Temp\tmppmo29hsd\assets


INFO:tensorflow:Assets written to: C:\Users\XZHANG~1\AppData\Local\Temp\tmppmo29hsd\assets
